In [1]:
!git clone https://github.com/bCom5/Awesome-Deep-Neural-Network-Compression.git
%cd Awesome-Deep-Neural-Network-Compression

Cloning into 'Awesome-Deep-Neural-Network-Compression'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 284 (delta 50), reused 70 (delta 27), pack-reused 189
Receiving objects: 100% (284/284), 1.66 MiB | 4.17 MiB/s, done.
Resolving deltas: 100% (144/144), done.
/content/Awesome-Deep-Neural-Network-Compression


In [2]:
%cd DoReFa
import sys
sys.path.append('../')
# includes utils?


/content/Awesome-Deep-Neural-Network-Compression/DoReFa


In [0]:
"""
Experimental implementation of Trained Ternary Quantization
"""

import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import shutil
import time

import torch
import torch.nn as nn
import torch.optim as optim

from utils.dataset import get_dataloader, get_lmdb_imagenet
from models.quantized_resnet import resnet20_cifar, resnet20_stl
from utils.quantize import test
from utils.train import accuracy
from utils.recorder import Recorder

import argparse

In [4]:
def boolean_string(s):
    if s not in {'False', 'True'}:
        raise ValueError('Not a valid boolean string')
    return s == 'True'

parser = argparse.ArgumentParser(description='Approximation Training')
parser.add_argument('--model', '-m', type=str, default='ResNet20', help='Model Arch')
parser.add_argument('--dataset', '-d', type=str, default='CIFAR10', help='Dataset')
parser.add_argument('--optimizer', '-o', type=str, default='Adam', help='Optimizer Method')
parser.add_argument('--quantized', '-q', type=str, default='dorefa', help='What type of quantization used')
parser.add_argument('--bitW', '-bw', type=int, default=1, help='Number of quantization bits')
parser.add_argument('--exp_spec', '-e', type=str, default='', help='Experiment Specification')
parser.add_argument('--init_lr', '-lr', type=float, default=1e-3, help='Initial Learning rate')
parser.add_argument('--n_epoch', '-n', type=int, default=100, help='Maximum training epochs')
parser.add_argument('--batch_size', '-bs', type=int, default=128, help='Batch size')

_StoreAction(option_strings=['--batch_size', '-bs'], dest='batch_size', nargs=None, const=None, default=128, type=<class 'int'>, choices=None, help='Batch size', metavar=None)

## Set args

In [0]:
bits = '8'
bits_lst = ['32', '24', '16', '8', '1']
# maybe 4

args = parser.parse_args(['-m', 'ResNet20', '-d', 'CIFAR10', '-q', 'dorefa', '-bw', bits])

In [6]:
# ------------------------------------------
use_cuda = torch.cuda.is_available()
model_name = args.model
dataset_name = args.dataset
quantized = args.quantized
MAX_EPOCH = args.n_epoch
optimizer_type = args.optimizer # ['SGD', 'SGD-M', 'adam']
dataset_type = 'large' if dataset_name in ['ImageNet'] else 'small'
# lr_adjust = args.lr_adjust
batch_size = args.batch_size
bitW = args.bitW
# ------------------------------------------

print(args)
#input('Take a look')

Namespace(batch_size=128, bitW=8, dataset='CIFAR10', exp_spec='', init_lr=0.001, model='ResNet20', n_epoch=100, optimizer='Adam', quantized='dorefa')


## Run code

In [7]:


if dataset_name in ['CIFAR10', 'STL10']:
    num_classes = 10
    save_root = '../Results/%s-%s' % (model_name, dataset_name)
elif dataset_name == 'CIFAR100':
    num_classes = 100
    save_root = '../Results/%s-%s' % (model_name, dataset_name)
elif dataset_name == 'ImageNet':
    save_root = '../Results/%s' % model_name
else:
    raise NotImplementedError


###################
# Initial Network #
###################
if model_name == 'ResNet20':
    if dataset_name in ['CIFAR10', 'CIFAR100']:
        net = resnet20_cifar(num_classes=num_classes, bitW=bitW)
    elif dataset_name in ['STL10']:
        net = resnet20_stl(num_classes=num_classes, bitW=bitW)
    else:
        raise NotImplementedError
    pretrain_path = '%s/%s-%s-pretrain.pth' % (save_root, model_name, dataset_name)
else:
    raise NotImplementedError

# if pretrain
# net.load_state_dict(torch.load(pretrain_path), strict=False)

if use_cuda:
    net.cuda()

if optimizer_type == 'SGD-M':
    optimizer = optim.SGD(net.parameters(), lr=args.init_lr, momentum=0.9, weight_decay=5e-4)
elif optimizer_type == 'SGD':
    optimizer = optim.SGD(net.parameters(), lr=args.init_lr)
elif optimizer_type in ['adam', 'Adam']:
    optimizer = optim.Adam(net.parameters(), lr=args.init_lr)
else:
    raise NotImplementedError


################
# Load Dataset #
################
if dataset_name == 'ImageNet':
    try:
        train_loader = get_lmdb_imagenet('train', batch_size)
        test_loader = get_lmdb_imagenet('test', 100)
    except:
        train_loader = get_dataloader(dataset_name, 'train', batch_size)
        test_loader = get_dataloader(dataset_name, 'test', 100)
else:
    train_loader = get_dataloader(dataset_name, 'train', batch_size)
    test_loader = get_dataloader(dataset_name, 'test', 100)


####################
# Initial Recorder #
####################
SummaryPath = '%s/runs/%s-%s-%dbits' %(save_root, quantized, optimizer_type, bitW)
if args.exp_spec is not '':
    SummaryPath += ('-' + args.exp_spec)

print('Save to %s' %SummaryPath)

if os.path.exists(SummaryPath):
    print('Record exist, remove')
    input()
    shutil.rmtree(SummaryPath)
    os.makedirs(SummaryPath)
else:
    os.makedirs(SummaryPath)

Initial quantized CNN with bit 8
Initial quantized CNN with bit 8
Initial quantized CNN with bit 8
Initial quantized CNN with bit 8
Initial quantized CNN with bit 8
Initial quantized CNN with bit 8
Initial quantized CNN with bit 8
Initial quantized CNN with bit 8
Initial quantized CNN with bit 8
Initial quantized CNN with bit 8
Initial quantized CNN with bit 8
Initial quantized CNN with bit 8
Initial quantized CNN with bit 8
Initial quantized CNN with bit 8
Initial quantized CNN with bit 8
Initial quantized CNN with bit 8
Initial quantized CNN with bit 8
Initial quantized CNN with bit 8
Initial quantized CNN with bit 8
Initial quantized CNN with bit 8
Initial quantized CNN with bit 8
Initial quantized Linear with bit 8
[2020-03-14 21:20:09.610458] Loading train-None from CIFAR10



Number of training instances used: 50000
[DATA LOADING] Loading from CIFAR10-train-None finish. Number of images: 50000, Number of batches: 391
[2020-03-14 21:20:16.452211] Loading test-None from CIFAR10
Found CIFAR10 in /home/sinno/datasets/CIFAR10
Files already downloaded and verified
[DATA LOADING] Loading from CIFAR10-test-None finish. Number of images: 10000, Number of batches: 100
Save to ../Results/ResNet20-CIFAR10/runs/dorefa-Adam-8bits


In [9]:
recorder = Recorder(SummaryPath=SummaryPath, dataset_name=dataset_name)

for epoch in range(MAX_EPOCH):

    if recorder.stop:
        break

    net.train()
    end = time.time()

    recorder.reset_performance()

    print('Epoch: %d, lr: %e' %(epoch, optimizer.param_groups[0]['lr']))

    for batch_idx, (inputs, targets) in enumerate(train_loader):

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()

        optimizer.zero_grad()

        outputs = net(inputs, quantized=quantized)
        losses = nn.CrossEntropyLoss()(outputs, targets)
        losses.backward()

        optimizer.step()

        recorder.update(loss=losses.item(), acc=accuracy(outputs.data, targets.data, (1, 5)),
                        batch_size=outputs.shape[0], cur_lr=optimizer.param_groups[0]['lr'], end=end)

        recorder.print_training_result(batch_idx, len(train_loader))
        end = time.time()

    test_acc = test(net, quantized_type=quantized, test_loader=test_loader, dataset_name=dataset_name)

    recorder.update(loss=None, acc=test_acc, batch_size=0, end=None, is_train=False)
    # Adjust lr
    recorder.adjust_lr(optimizer=optimizer)

print('Best test acc: %s' %recorder.get_best_test_acc())
recorder.close()

Epoch: 0, lr: 1.000000e-04
 [==============>] | Loss: 0.032, Acc: 99.028% |  391/391 
 [==============>] | Test Acc: 89.880% 100/100 
Best test acc
Current smallest training loss: 12.530
Epoch: 1, lr: 1.000000e-04
 [==============>] | Loss: 0.032, Acc: 98.990% |  391/391 
 [==============>] | Test Acc: 89.990% 100/100 
Best test acc
Training loss: 12.631 [12.530], ascend count: 1
Epoch: 2, lr: 1.000000e-04
 [==============>] | Loss: 0.030, Acc: 99.092% |  391/391 
 [==============>] | Test Acc: 89.930% 100/100 
Current smallest training loss: 11.907
Epoch: 3, lr: 1.000000e-04
 [==============>] | Loss: 0.033, Acc: 99.006% |  391/391 
 [==============>] | Test Acc: 89.890% 100/100 
Training loss: 12.719 [11.907], ascend count: 1
Epoch: 4, lr: 1.000000e-04
 [==============>] | Loss: 0.029, Acc: 99.156% |  391/391 
 [==============>] | Test Acc: 89.800% 100/100 
Current smallest training loss: 11.233
Epoch: 5, lr: 1.000000e-04
 [==============>] | Loss: 0.031, Acc: 99.032% |  391/391 
 [=

In [10]:
# model.cpu();
torch.save(net.state_dict(), 'resnet20_8bit_accuracy90_2_date20_03_14.pytorch_state')

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!mv resnet20_8bit_accuracy90_2_date20_03_14.pytorch_state /content/gdrive/My\ Drive/models/resnet20_8bit_accuracy90_2_date20_03_14.pytorch_state